In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.spatial import distance
import xml.etree.ElementTree as ET
from shapely.geometry import Polygon, LineString
import pandas as pd
#from src.generate_traffic_data import clean_carla, spawn_scene, run_scene
import carla
import os
os.getcwd()
from datetime import datetime
import time
import random
from tqdm import tqdm
import networkx as nx
import numpy as np
import pickle
from graph_creator.MapGraph import MapGraph
from graph_creator.utilities import make_node_edge_df
from graph_creator.SubgraphIsomorphismChecker import IsomorphicGrapCoverageCounter

os.chdir('../..')
os.getcwd()

#from graph_creator.MapGraph import MapGraph
#from graph_creator.ActorGraph import ActorGraph

from graph_creator.plot_graphs import plot_lane_map_advanced, add_actors_to_map, add_actor_edges_to_map

from glob import glob
import pickle
import networkx as nx
import random
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

graph_paths = glob("/home/tmuehlen/repos/graph_coverage/actor_graphs/carla_w_intersecion/*.pkl")
len(graph_paths)

12360

### Start comparing graphs by randomly picking some:

In [2]:
# Define a random set of coverage situations with 3 nodes each
# This still needs to be replaced by more strategically selected graphs

#idxs = [100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000]
idxs = random.sample(list(range(len(graph_paths))), 10)

coverage_graphs = {}
cov_data = {}

for idx in idxs:
    graph_path = graph_paths[idx]
    with open(graph_path, "rb") as file:
        ag_nx = pickle.load(file)
    #random.sample(ag_nx.nodes, 3)
    if ag_nx.number_of_nodes() > 2:
        coverage_graphs[idx] = ag_nx.subgraph(random.sample(list(ag_nx.nodes), 3))
        cov_data[idx] = []

cov_data["degree"] = []
cov_data["density"] = []
cov_data["diameter"] = []


In [ ]:
# 

In [5]:
isom_cov_counter = IsomorphicGrapCoverageCounter(coverage_graphs, graph_paths[:100], node_match = ["actor_type", "lane_change", "is_on_intersection"], edge_match = ["edge_type"])
isom_cov_counter.count_isomorphic_graphs()
cov_data_df = isom_cov_counter.cov_data_df

100%|██████████| 100/100 [00:00<00:00, 563.76it/s]


In [6]:

list(cov_data_df.columns[:10])


[6734, 11012, 11540, 'degree', 'density', 'diameter']

In [9]:
cov_data_df.mean()

6734        0.320000
11012       0.410000
11540       0.320000
degree      2.756719
density     0.648132
diameter    3.480000
dtype: float64

In [8]:
#cov_data_df.groupby(list(cov_data_df.columns[:11]), as_index=False).size()


In [ ]:
# sns.pairplot(data=cov_data_df)
# plt.show()


### define some situations manually

In [ ]:
#graph_1 = pca_emb_df.graph_path.iloc[0].split("/")[-1].split(".pkl")[0].split("graph_")[1]
#scn_1 = graph_1[:26]

In [ ]:
idx = 10
graph_path = graph_paths[idx]
print(graph_path)
with open(graph_path, "rb") as file:
    ag_nx = pickle.load(file)


graph_1 = graph_path.split("/")[-1].split(".pkl")[0].split("graph_")[1]
scn_1 = graph_1[:26]

g_map_1 = MapGraph()
g_map_1.read_graph_from_file(f'/home/tmuehlen/repos/graph_coverage/carla_new/data/scene_{scn_1}_map_graph.pickle')
g_map_1 = g_map_1.graph

fig, ax1 = plt.subplots(1, 1, figsize=(20, 20))
fig, ax1 = plot_lane_map_advanced(g_map_1, figsize=None, color_by_length=True, cmap='plasma', show_labels=False, fig = fig, ax = ax1)
fig, ax1 = add_actors_to_map(fig, ax1, ag_nx, actor_size=20, show_actor_labels=True)
#fig, ax1 = add_actor_edges_to_map(fig, ax1, ag_nx)
ax1.set_title(f'Scene {scn_1}')


In [ ]:
graph_path

In [ ]:
# 3 vehicles in a row:
coverage_graphs = {}
cov_data = {}

#idx = 10
graph_path = graph_paths[idx]
print(graph_path)

In [ ]:
df_nodes, df_edges = make_node_edge_df(ag_nx)
display(df_nodes)
display(df_edges)


In [ ]:

graph_path = '/home/tmuehlen/repos/graph_coverage/actor_graphs/carla_w_intersecion/graph_2025-09-14 22:08:32.660591_11_0.pkl'
with open(graph_path, "rb") as file:
    ag_nx = pickle.load(file)

graph_1 = graph_path.split("/")[-1].split(".pkl")[0].split("graph_")[1]
scn_1 = graph_1[:26]

g_map_1 = MapGraph()
g_map_1.read_graph_from_file(f'/home/tmuehlen/repos/graph_coverage/carla/data/scene_{scn_1}_map_graph.pickle')
g_map_1 = g_map_1.graph

coverage_graphs["three_vehicles_same_lane_in_a_row_no_lane_change"] = ag_nx.subgraph(["6724", "6692", "6726"])

scene_name = "three_vehicles_same_lane_in_a_row_no_lane_change"

df_nodes, df_edges = make_node_edge_df(coverage_graphs[scene_name])

# df_nodes = pd.DataFrame.from_dict(dict(coverage_graphs[scene_name].nodes(data=True)), orient='index')
# df_nodes.reset_index(inplace=True)
# df_nodes.rename(columns={'index': 'node_id'}, inplace=True)
# df_edges = pd.DataFrame([(u, v, d) for u, v, d in coverage_graphs[scene_name].edges(data=True)], columns=['source', 'target', 'attributes'])
# df_edges = pd.concat([df_edges[['source', 'target']], pd.json_normalize(df_edges['attributes'])], axis=1)
display(df_nodes)
display(df_edges)


In [ ]:
tmp = ag_nx.subgraph(["6724", "6692", "6726"])
tmp.nodes["6724"]["lane_change"] = True


# Notebook junkyard

In [ ]:
# class IsomorphicGrapCoverageCounter:
#     def __init__(self, coverage_graphs: dict, graph_paths: list, node_match: list, edge_match: list):
#         """
#         Initialize the IsomorphicGrapCoverageCounter.

#         Args:
#             coverage_graphs: Dictionary of coverage graphs.
#             graph_paths: List of graph paths.
#             node_match: List of node match attributes.
#             edge_match: List of edge match attributes.
#         """
#         self.coverage_graphs = coverage_graphs
#         self.graph_paths = graph_paths
#         self.node_match = node_match
#         self.edge_match = edge_match
#         self.cov_data = {}
#         for scen_name in self.coverage_graphs.keys():
#             self.cov_data[scen_name] = []

#         self.cov_data["degree"] = []
#         self.cov_data["density"] = []
#         self.cov_data["diameter"] = []


#     def count_isomorphic_graphs(self):
#         for graph_path in tqdm(self.graph_paths):
#             with open(graph_path, "rb") as file:
#                 ag_nx = pickle.load(file)

#             for key in self.coverage_graphs.keys():
#                 GM = nx.algorithms.isomorphism.DiGraphMatcher(
#                     ag_nx, self.coverage_graphs[key],
#                     node_match=nx.algorithms.isomorphism.categorical_node_match(self.node_match, [None] * len(self.node_match)),
#                     edge_match=nx.algorithms.isomorphism.categorical_edge_match(self.edge_match, [None] * len(self.edge_match))
#                 )
#                 self.cov_data[key].append(GM.subgraph_is_isomorphic())
            
#             self.cov_data["degree"].append(sum(dict(ag_nx.degree()).values()) / len(ag_nx.nodes()))
#             self.cov_data["density"].append(nx.density(ag_nx))
#             self.cov_data["diameter"].append(nx.diameter(ag_nx))

#         self.cov_data_df = pd.DataFrame(self.cov_data)
